# Profitable App Profiles for the App Store and Google Play Markets
Our aim in this project is to find mobile app profiles that are profitable for the App Store and Google Play markets. We're working as data analysts for a company that builds Android and iOS mobile apps, and our job is to enable our team of developers to make data-driven decisions with respect to the kind of apps they build.

At our company, we only build apps that are free to download and install, and our main source of revenue consists of in-app ads. This means that our revenue for any given app is mostly influenced by the number of users that use our app. Our goal for this project is to analyze data to help our developers understand what kinds of apps are likely to attract more users.

# Opening and Exploring the Data
As of September 2018, there were approximately 2 million iOS apps available on the App Store, and 2.1 million Android apps on Google Play.

Collecting data for over four million apps requires a significant amount of time and money, so we'll try to analyze a sample of data instead. To avoid spending resources with collecting new data ourselves, we should first try to see whether we can find any relevant existing data at no cost. Luckily, these are two data sets that seem suitable for our purpose:

* [A data set](https://www.kaggle.com/lava18/google-play-store-apps/home) containing data about approximately ten thousand Android apps from Google Play
* [A data set](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps/home) containing data about approximately seven thousand iOS apps from the App Store
Let's start by opening the two data sets and then continue with exploring the data.

In [4]:
from csv import reader

file_opened = open("data_sets/AppleStore.csv", encoding='utf8')
file_readed = list(reader(file_opened))
apple_date = file_readed[1:]
apple_headers = file_readed[0]

In [5]:
from csv import reader

file_opened = open("data_sets/googleplaystore.csv", encoding='utf8')
file_readed = list(reader(file_opened))
google_date = file_readed[1:]
google_headers = file_readed[0]

To make it easier to explore the two data sets, we'll first write a function named explore_data() that we can use repeatedly to explore rows in a more readable way. We'll also add an option for our function to show the number of rows and columns for any data set.

In [6]:
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print("%s\n" % row)

    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))

In [11]:
explore_data(apple_headers, 0, len(apple_headers), False)
explore_data(apple_date, 0, 1, True)



id

track_name

size_bytes

currency

price

rating_count_tot

rating_count_ver

user_rating

user_rating_ver

ver

cont_rating

prime_genre

sup_devices.num

ipadSc_urls.num

lang.num

vpp_lic

['1', '281656475', 'PAC-MAN Premium', '100788224', 'USD', '3.99', '21292', '26', '4', '4.5', '6.3.5', '4+', 'Games', '38', '5', '10', '1']

Number of rows: 7197
Number of columns: 17


In [10]:
explore_data(google_headers, 0, len(google_headers), False)
explore_data(google_date, 0, 1, True)

App

Category

Rating

Reviews

Size

Installs

Type

Price

Content Rating

Genres

Last Updated

Current Ver

Android Ver

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']

Number of rows: 10841
Number of columns: 13
